In [ ]:
### Unique Identifier Extraction

(currently notes)

Anecdata - folks don't really comply with the URN spec but will include URN-like things.

I don't have any plan for identifying things tagged as "cool" URIs but aren't full URLs. 


####URNs We Can't Identify

Using some regex that is not really functional:

```
^([a-z0-9.#]{0,}:([a-z0-9][a-z0-9.-]{0,31}):[a-z0-9A-Z_()+,-.:=@;$!*\'%/?#\[\]]+)
```

and that is expecting 3 parts.

Not a URN (not saying they should be, either):

 - nlm:nlmuid-67840340R-bk
 - geonode:combined_tasks
 - gov.noaa.ngdc.stp.indices:G00587
 - gov.noaa.nmfs.inport:18623

 
 
Things that are a URN according to the regex:

 - urn:nbn:nl:ui:13-01hb-3b (extracted from a rule set)
 - oai:easy.dans.knaw.nl:easy-dataset:53685 (coded as a url)
 - oai:www.mpi.nl:MPI110411
 - clarin.eu:cr1:p_1271859438204
 - urn:uuid:6426c02b-f2b1-4326-a767-2384c303faf3
 - oai:oai.datacite.org:1979342
 
 
 
 

####More generally things that are awkward

A piece of text prior to identifier extraction. 

St1958-03-31T00:00:00-111.155555685007http://services.azgs.az.gov/OnlineAccessMineFiles/M-R/MilewidePima15.pdfDocument

It's composed of a timestamp, a URL and a file type. Using the regex, we would extract more text than just the URL for the URL identifier (so it would keep the ".pdfDocument".

### About the Identifiers

There are a couple of layers of extraction. There's a set of unqualified paths (they don't need to be the full path) for known identifier locations. We can't assume that these will conform to a unique identifier specfication or structure but is being provided as some identifier by the publisher. We are taking that in good faith.


**What it extracts**

URLS: 

http://vmo.igpp.ucla.edu/data1/SAMBA/2004/Feb/S04043VLD1s.asc
https://workspace.nottingham.ac.uk/pages/metadata/viewxml.action?pageId=132225079
ftp://ftp.nmh.ac.uk/wdc/obsdata/1minval
gsiftp://vetsman.ucar.edu:2811/


Handles:

http://hdl.handle.net/10022/AC:P:449


DOIs:

http://dx.doi.org/10.7916/D85B019G
10.7916/D85B019G
doi:10.5284/1017355


Cool URIs:

(a cool URI is pretty broad here - alphanumerics between forward slashes and a couple of select punctuation chars)

samos/data/research/WTEP/2009/WTEP_20090509v30001.nc
/opendap/hyrax/TRMM_L2/TRMM_2A23/2008/087/2A23.20080327.59048.7.HDF.Z

URNs:

SC:MOD17A2.005:2042282974
urn:uuid:c813a42d-0385-4cf8-b415-3e7c3783afef


UUIDs:

0d0a763e-76af-4e24-a5fd-c3930de5bed8

MD5s: 

cbb52feebe70d4d3821727b8d5dda65c


Identifier from a defined xpath rule:

gov.noaa.ngdc.sem.goes_xrs_g00036
freezingHeightIndex
V237


In [ ]:
%matplotlib inline
import pandas as pd
import json as js  # name conflict with sqla
import sqlalchemy as sqla
from sqlalchemy.orm import sessionmaker
from IPython.display import display
from IPython.display import Image

In [ ]:
# grab the clean text from the rds
with open('../local/big_rds.conf', 'r') as f:
    conf = js.loads(f.read())

# our connection
engine = sqla.create_engine(conf.get('connection'))

In [ ]:
# responses with one identifier that is the harveset url.
sql = """
with j as (
	with i as 
	(
		select response_id, count(potential_identifier) as potentials
		from unique_identifiers
		group by response_id
	)
	select r.id, r.source_url, i.potentials, u.potential_identifier, u.match_type
	from responses r 
		join i on i.response_id = r.id
		join unique_identifiers u on u.response_id = r.id
	where i.potentials = 1 and u.match_type = 'url'
)
select j.id, j.source_url
from j
where j.source_url = j.potential_identifier;

"""